In [2]:
import os
import sys
from pathlib import Path
sys.path.append("../")

In [3]:
import numpy as np

def extract_patch(image: np.ndarray, coords, cut_size: int = 9):
    coord_x, coord_y, coord_z = coords
    patch_size = cut_size * 2 + 1

    # Define start and end indices for the patch
    start_x, end_x = max(coord_x - cut_size, 0), min(coord_x + cut_size, image.shape[2] - 1)
    start_y, end_y = max(coord_y - cut_size, 0), min(coord_y + cut_size, image.shape[1] - 1)
    start_z, end_z = max(coord_z - cut_size, 0), min(coord_z + cut_size, image.shape[0] - 1)

    # Initialize the patch with a fill value
    fill_value = -50  # TODO: Adjust based on your data
    patch = np.full((patch_size, patch_size, patch_size), fill_value, dtype=image.dtype)


    # Extract the valid region and place it into the patch
    patch[start_z - coord_z + cut_size:end_z - coord_z + cut_size + 1,
          start_y - coord_y + cut_size:end_y - coord_y + cut_size + 1,
          start_x - coord_x + cut_size:end_x - coord_x + cut_size + 1] = \
          image[start_z:end_z + 1, start_y:end_y + 1, start_x:end_x + 1]

    return patch

In [4]:
arr = np.random.rand(512, 512, 400)
extract_patch(arr, coords=(100, 100, 100), cut_size=9).shape

(19, 19, 19)

In [5]:
import numpy as np
import cv2
from skimage import measure

def crop_heart(input_arr):
    """
    Improved function to segment the heart region using thresholding and morphological operations.
    :param input_arr: 3D array representing the CCTA scan.
    :return: Cropped heart data and bounding box coordinates.
    """
    print(f"Shape of input array is: {input_arr.shape}")

    src_array = input_arr.astype(np.float32)
    z, w, h = src_array.shape
    new_arr = np.full_like(src_array, -1000)  # Use np.full_like for efficiency

    # Initialize variables for calculating mean bounding box
    #sum_minr, sum_minc, sum_maxr, sum_maxc = 0, 0, 0, 0
    minrs, mincs, maxrs, maxcs = [], [], [], []

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    for k in range(z):
        image = src_array[k]
        ret, thresh = cv2.threshold(image, 20, 650, cv2.THRESH_BINARY)
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=4)

        label_opening = measure.label(opening)
        regionprops = measure.regionprops(label_opening)

        if regionprops:
            # Find the largest region, assuming it's the heart
            largest_region = max(regionprops, key=lambda x: x.area)
            minr, minc, maxr, maxc = largest_region.bbox

            # Update the new array and bounding box sums
            new_arr[k, minr:maxr, minc:maxc] = src_array[k, minr:maxr, minc:maxc]
            # sum_minr += minr
            # sum_minc += minc
            # sum_maxr += maxr
            # sum_maxc += maxc
            minrs.append(minr)
            mincs.append(minc)
            maxrs.append(maxr)
            maxcs.append(maxc)

    # Calculate mean bounding box coordinates
    # mean_minr = sum_minr // z
    # mean_minc = sum_minc // z
    # mean_maxr = sum_maxr // z
    # mean_maxc = sum_maxc // z

    # Calculate the index for the upper third
    upper_third_index = new_arr.shape[0] // 3
    upper_third_index = int(upper_third_index - (upper_third_index*0.15))

    # Extract the upper third of the CCTA scan
    upper_third_new_arr = new_arr[upper_third_index*2:]
    
    # discard measures associated to slices in z axis we just discarded
    
    
    min_minr = np.array(minrs)[upper_third_index*2:].min()
    min_minc = np.array(mincs)[upper_third_index*2:].min()
    max_maxr = np.array(maxrs)[upper_third_index*2:].max()
    max_maxc = np.array(maxcs)[upper_third_index*2:].max()
    minz = upper_third_index
    maxz = new_arr.shape[0]
    
    return upper_third_new_arr, min_minc, min_minr, max_maxc, max_maxr, minz, maxz

In [121]:
from landmark_localization.utils.misc import get_all_hsr_file_paths

In [6]:
data_path = Path("/Users/francescopisu/Workspace/Research/Projects/CoroCTAiomics/notebooks/data/10000AD9.nii.gz")

In [7]:
import SimpleITK as sitk

In [8]:
image_sitk = sitk.ReadImage(data_path.as_posix())
origin = image_sitk.GetOrigin()
spacing = image_sitk.GetSpacing()
direction = image_sitk.GetDirection()
image_arr = sitk.GetArrayFromImage(image=image_sitk)

In [9]:
import matplotlib.pyplot as plt

crop, minc, minr, maxc, maxr, minz, maxz = crop_heart(image_arr)

Shape of input array is: (283, 512, 512)


In [10]:
minc, minr, maxc, maxr, minz, maxz

(18, 38, 349, 403, 79, 283)

In [130]:
crop_sitk = sitk.GetImageFromArray(crop)
crop_sitk.SetOrigin(origin)
crop_sitk.SetSpacing(spacing)
crop_sitk.SetDirection(direction)
sitk.WriteImage(crop_sitk, "../results/10000AD9/crop.nii.gz")

In [139]:
# crop using the min and max row and columns indices
crop2 = image_arr[minz:maxz+1, 
                  minr:maxr+1,
                  minc:maxc+1]

crop2_sitk = sitk.GetImageFromArray(crop2)
crop2_sitk.SetOrigin(origin)
crop2_sitk.SetSpacing(spacing)
crop2_sitk.SetDirection(direction)
sitk.WriteImage(crop2_sitk, "../results/10000AD9/crop2.nii.gz")